# Bayesian Personalized Ranking (BPR)

In [38]:
import sys
import cornac
import pandas as pd
import numpy as np

from recommenders.evaluation.python_evaluation import map, ndcg_at_k, precision_at_k, recall_at_k, diversity, novelty
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")

## Variables

In [24]:
# top k items to recommend
TOP_K = 50

# Model parameters
NUM_FACTORS = 200
NUM_EPOCHS = 100

# Column names for the dataset
COL_USER = "user_id"
COL_TRACK = "track_id"
COL_COUNT = "playcount"

## Load and split data

### Load data

In [25]:
# Read from file
test_listening_history = pd.read_csv(header=0, delimiter="\t", filepath_or_buffer="../remappings/data/dataset/test_listening_history_OverEqual_50_Interactions.txt")
train_listening_history = pd.read_csv(header=0, delimiter="\t", filepath_or_buffer="../remappings/data/dataset/train_listening_history_OverEqual_50_Interactions.txt")

# Change columns to correct place (user_id, track_id, playcount)
track_test = test_listening_history["track_id"]
user_test = test_listening_history["user_id"]

track_train = train_listening_history["track_id"]
user_train = train_listening_history["user_id"]

test_listening_history["track_id"] = user_test
test_listening_history["user_id"] = track_test

train_listening_history["track_id"] = user_train
train_listening_history["user_id"] = track_train

test_listening_history.columns = [COL_USER, COL_TRACK, COL_COUNT]
train_listening_history.columns = [COL_USER, COL_TRACK, COL_COUNT]

### Split data

In [26]:
train, test = train_listening_history, test_listening_history

# Set the alpha value for the confidence transformation
alpha = 1

# Transform playcount to confidence in the training data only
train["confidence"] = 1 + alpha * np.log(1 + train[COL_COUNT])

## Build a Cornac Dataset

To work with models implemented in Cornac, we need to construct an object from [Dataset](https://cornac.readthedocs.io/en/latest/data.html#module-cornac.data.dataset) class.

Dataset Class in Cornac serves as the main object that the models will interact with.  In addition to data transformations, Dataset provides a bunch of useful iterators for looping through the data, as well as supporting different negative sampling techniques.

In [27]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

## Train the BPR model

In [28]:
bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED)

In [ ]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))

## Prediction and Evaluation

In [ ]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, train, usercol=COL_USER, itemcol=COL_TRACK, remove_seen=True)
print("Took {} seconds for prediction.".format(t))

all_predictions.head()

In [31]:
# Sort by 'user' and 'prediction' in descending order
all_prediction_sorted = all_predictions.sort_values(by=[COL_USER, 'prediction'], ascending=[True, False])

# Select the top k predictions for each user
top_k_rec = all_prediction_sorted.groupby(COL_USER).head(TOP_K)

In [ ]:
eval_map = map(test, top_k_rec, 
               col_user=COL_USER, 
               col_item=COL_TRACK, 
               col_prediction='prediction', 
               k=TOP_K,
               relevancy_method=None)

eval_ndcg = ndcg_at_k(test, top_k_rec, 
                      col_user=COL_USER, 
                      col_item=COL_TRACK, 
                      col_rating=COL_COUNT, 
                      col_prediction='prediction', 
                      k=TOP_K, 
                      relevancy_method=None)

eval_precision = precision_at_k(test, top_k_rec, 
                                col_user=COL_USER, 
                                col_item=COL_TRACK, 
                                col_prediction='prediction', 
                                k=TOP_K, 
                                relevancy_method=None)

eval_recall = recall_at_k(test, top_k_rec, 
                          col_user=COL_USER, 
                          col_item=COL_TRACK, 
                          col_prediction='prediction', 
                          k=TOP_K, 
                          relevancy_method=None)

eval_diversity = diversity(train_df=train,
                           reco_df=top_k_rec,
                           col_user=COL_USER,
                           col_item=COL_TRACK)

eval_novelty = novelty(train_df=train,
                       reco_df=top_k_rec,
                       col_user=COL_USER,
                       col_item=COL_TRACK)

# Print evaluation metrics, including diversity
print("Precision@K Spark:\t%f" % eval_precision,
      "Recall@K Spark:\t%f" % eval_recall,
      "NDCG Spark:\t%f" % eval_ndcg,
      "Diversity Spark:\t%f" % eval_diversity,
      "Novelty Spark:\t%f" % eval_novelty, sep='\n')